In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, udf
from pyspark.sql.functions import sum,avg,max,min,mean,count
from pyspark.sql.types import IntegerType, DoubleType
spark = SparkSession.builder.appName("Spark DataFrames").getOrCreate()

In [0]:
df = spark.read.options(inferSchema="True",header="True",delimiter =',').csv("/FileStore/tables/OfficeDataProject.csv")
df.show()
df.printSchema()

+-----------+-------------------+----------+-----+------+---+-----+
employee_id| employee_name|department|state|salary|age|bonus|
+-----------+-------------------+----------+-----+------+---+-----+
 1000| Nitz Leif| Marketing| CA| 6131| 26| 543|
 1001| Melissia Dedman| Finance| AK| 4027| 43| 1290|
 1002| Rudolph Barringer| HR| LA| 3122| 43| 1445|
 1003| Tamra Amber| Accounts| AK| 5717| 47| 1291|
 1004| Mullan Nitz|Purchasing| CA| 5685| 34| 1394|
 1005| Zollner Karie| Accounts| CA| 2843| 27| 1078|
 1006|Kaczorowski Zollner| Sales| CA| 7201| 21| 1834|
 1007| Nakano Locust| Marketing| LA| 3444| 23| 1823|
 1008| Recalde Kensinger| Accounts| LA| 3704| 48| 1330|
 1009| Imai Hallie| Accounts| AK| 5061| 38| 1557|
 1010| Debroah Gallman| Accounts| NY| 9308| 35| 817|
 1011| Barringer Escoto|Purchasing| WA| 1685| 49| 1706|
 1012| Soules Coogan| Accounts| AK| 8330| 43| 1914|
 1013| Luisa Suzanne| Accounts| CA| 1151| 37| 1095|
 1014| Marvis Cobian|Purchasing| NY| 5061| 41| 1765|
 1015| Cobian Kensinger| Sales| LA| 1983| 21| 632|
 1016| Gilma Margret| Marketing| CA| 2919| 45| 1762|
 1017| Ellingsworth Ilana| Accounts| WA| 9614| 26| 1964|
 1018| Vankirk Jacquelyne|Purchasing| NY| 8636| 47| 1192|
 1019| Zollner Juliana| HR| NY| 9739| 30| 1119|
+-----------+-------------------+----------+-----+------+---+-----+
only showing top 20 rows

root
-- employee_id: integer (nullable = true)
-- employee_name: string (nullable = true)
-- department: string (nullable = true)
-- state: string (nullable = true)
-- salary: integer (nullable = true)
-- age: integer (nullable = true)
-- bonus: integer (nullable = true)

In [0]:
# 1. Print the total number of employees in the company
df.count()

Out[4]: 1000

In [0]:
# 2. Print the total number os departments in the company
df.select(col("department")).distinct().count()
#df.dropDuplicates(["department"]).count()

Out[5]: 6

In [0]:
# 3. Print the department names of the company
df.select(col("department")).distinct().show()
#df.select("department").dropDuplicates(["department"]).show()

+----------+
department|
+----------+
 Sales|
 HR|
 Finance|
Purchasing|
 Marketing|
 Accounts|
+----------+

In [0]:
# 4. Print the total number of employees in each department
df.groupBy(df.department).agg(count("*").alias("total_employees")).show()

+----------+---------------+
department|total_employees|
+----------+---------------+
 Sales| 169|
 HR| 171|
 Finance| 162|
Purchasing| 166|
 Marketing| 170|
 Accounts| 162|
+----------+---------------+

In [0]:
# 5. Print the total number of employees in each state
df.groupBy(df.state).agg(count("*").alias("total_employees")).show()

+-----+---------------+
state|total_employees|
+-----+---------------+
 LA| 205|
 CA| 205|
 WA| 208|
 NY| 173|
 AK| 209|
+-----+---------------+

In [0]:
# 6. Print the total number of employees in each state in each department
df.groupBy(df.state,df.department).agg(count("*").alias("total_employees")).show()

+-----+----------+---------------+
state|department|total_employees|
+-----+----------+---------------+
 CA| Sales| 42|
 CA| Marketing| 33|
 NY| Accounts| 34|
 NY| Sales| 27|
 CA| Finance| 35|
 CA| Accounts| 35|
 CA|Purchasing| 32|
 WA| HR| 47|
 AK|Purchasing| 30|
 WA| Accounts| 27|
 WA|Purchasing| 38|
 AK| Sales| 38|
 AK| Accounts| 37|
 WA| Marketing| 39|
 LA| HR| 41|
 LA| Sales| 35|
 AK| HR| 25|
 LA| Finance| 29|
 AK| Finance| 37|
 LA|Purchasing| 45|
+-----+----------+---------------+
only showing top 20 rows

In [0]:
# 7. Print the minimum and maximum salaries in each department and sort salaries in ascending order
df.groupBy(df.department).agg(min("salary").alias("min_salary"),max("salary").alias("max_salary")).sort(col("min_salary").asc(),col("max_salary").asc()).show()

+----------+----------+----------+
department|min_salary|max_salary|
+----------+----------+----------+
 Finance| 1006| 9899|
 Accounts| 1007| 9890|
 HR| 1013| 9982|
 Marketing| 1031| 9974|
 Sales| 1103| 9982|
Purchasing| 1105| 9985|
+----------+----------+----------+

In [0]:
# 8. Print the names of employees working in NY state under Finance epartment whose bonuses are greater than the average bonuses of employees in NY state
def state_avg_salary(state):
  return df.filter(df.state==state).groupBy(df.state).agg(avg(df.salary).alias("avg_salary")).select(col("avg_salary")).collect()[0][0]

In [0]:
state_avg_salary("NY")

Out[73]: 5226.086705202312

In [0]:
df.filter((df.state=="NY") & (df.department=="Finance") & (df.salary>state_avg_salary("NY"))).show()

+-----------+--------------------+----------+-----+------+---+-----+
employee_id| employee_name|department|state|salary|age|bonus|
+-----------+--------------------+----------+-----+------+---+-----+
 1079| Grigg Debroah| Finance| NY| 5728| 28| 1146|
 1087| Leif Lemaster| Finance| NY| 8642| 45| 1782|
 1100|Ellingsworth Meli...| Finance| NY| 7845| 32| 1358|
 1180| Juliana Grigg| Finance| NY| 8178| 42| 1617|
 1286| Kohn Vankirk| Finance| NY| 5754| 39| 1229|
 1301| Melissia Katlyn| Finance| NY| 5731| 26| 525|
 1335| Deandre Melissia| Finance| NY| 8898| 32| 513|
 1342| Phylicia Antonina| Finance| NY| 8626| 30| 1857|
 1509| Ellingsworth Nitz| Finance| NY| 7234| 45| 1004|
 1660| Melissia Jere| Finance| NY| 9201| 21| 1533|
 1737| Yukiko Kreamer| Finance| NY| 7396| 20| 1332|
 1738| Nena Kensinger| Finance| NY| 8534| 41| 1610|
 1780| Margret Reardon| Finance| NY| 9677| 47| 567|
 1845| Stanford Luisa| Finance| NY| 8499| 47| 595|
 1854| Kensinger Benz| Finance| NY| 7739| 28| 611|
+-----------+--------------------+----------+-----+------+---+-----+

In [0]:
# 9. Raise the salaries $500 of all employees whose age is greater than 45
def raise_500(x,y):
  if x >45:
    return y + 500
  return y
  
raise_500UDF = udf(lambda x,y: raise_500(x,y), IntegerType())

In [0]:
df2 = df.withColumn("salary", raise_500UDF(col("age"),col("salary")))
df2.filter(col("age")>45).show()

+-----------+------------------+----------+-----+------+---+-----+
employee_id| employee_name|department|state|salary|age|bonus|
+-----------+------------------+----------+-----+------+---+-----+
 1003| Tamra Amber| Accounts| AK| 6217| 47| 1291|
 1008| Recalde Kensinger| Accounts| LA| 4204| 48| 1330|
 1011| Barringer Escoto|Purchasing| WA| 2185| 49| 1706|
 1018|Vankirk Jacquelyne|Purchasing| NY| 9136| 47| 1192|
 1025| Dionne Lemaster| Sales| AK| 5634| 48| 1356|
 1030| Trena Benz| Accounts| NY| 4876| 49| 1624|
 1039| Dynes Katlyn| Accounts| AK| 3539| 48| 834|
 1058| Clune Norene| Finance| AK| 2105| 49| 801|
 1074| Rocha Dionne| Accounts| CA| 3970| 49| 706|
 1088| Imai Locust| Sales| NY| 10482| 49| 1809|
 1090| Clemencia Rudolph| Finance| NY| 1796| 50| 1209|
 1099| Zollner Marvis|Purchasing| CA| 4730| 50| 951|
 1118| Kohn Antonina| Sales| CA| 9811| 48| 1229|
 1126| Norene Mayeda| Accounts| LA| 3600| 50| 1333|
 1136| Jaclyn Baros| Sales| LA| 2199| 48| 1094|
 1164| Gallman Nakano| Sales| CA| 7313| 47| 857|
 1165| Luisa Grigg|Purchasing| AK| 9797| 48| 1545|
 1169| Kreamer Shandra| HR| WA| 5468| 46| 889|
 1170| Fender Coogan| Marketing| AK| 1651| 50| 1845|
 1183| Lonergan Bergren| HR| NY| 8675| 46| 1730|
+-----------+------------------+----------+-----+------+---+-----+
only showing top 20 rows

In [0]:
df.filter(col("age")>45).show()

+-----------+------------------+----------+-----+------+---+-----+
employee_id| employee_name|department|state|salary|age|bonus|
+-----------+------------------+----------+-----+------+---+-----+
 1003| Tamra Amber| Accounts| AK| 5717| 47| 1291|
 1008| Recalde Kensinger| Accounts| LA| 3704| 48| 1330|
 1011| Barringer Escoto|Purchasing| WA| 1685| 49| 1706|
 1018|Vankirk Jacquelyne|Purchasing| NY| 8636| 47| 1192|
 1025| Dionne Lemaster| Sales| AK| 5134| 48| 1356|
 1030| Trena Benz| Accounts| NY| 4376| 49| 1624|
 1039| Dynes Katlyn| Accounts| AK| 3039| 48| 834|
 1058| Clune Norene| Finance| AK| 1605| 49| 801|
 1074| Rocha Dionne| Accounts| CA| 3470| 49| 706|
 1088| Imai Locust| Sales| NY| 9982| 49| 1809|
 1090| Clemencia Rudolph| Finance| NY| 1296| 50| 1209|
 1099| Zollner Marvis|Purchasing| CA| 4230| 50| 951|
 1118| Kohn Antonina| Sales| CA| 9311| 48| 1229|
 1126| Norene Mayeda| Accounts| LA| 3100| 50| 1333|
 1136| Jaclyn Baros| Sales| LA| 1699| 48| 1094|
 1164| Gallman Nakano| Sales| CA| 6813| 47| 857|
 1165| Luisa Grigg|Purchasing| AK| 9297| 48| 1545|
 1169| Kreamer Shandra| HR| WA| 4968| 46| 889|
 1170| Fender Coogan| Marketing| AK| 1151| 50| 1845|
 1183| Lonergan Bergren| HR| NY| 8175| 46| 1730|
+-----------+------------------+----------+-----+------+---+-----+
only showing top 20 rows

In [0]:
# 10. Create a DF of all those employees whose age is greater than 45 and save them in a file
df2.filter(col("age")>45).write.mode("overwrite").options(header="True").csv("/FileStore/tables/MiniProject_DF/output")